import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [2]:
import findspark
findspark.init()

In [3]:
import pyspark
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import max, avg, min
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import when

create local SparkSession

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("My PySpark") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

read csv with inferschema

In [5]:
df = spark.read.csv("ds_salaries.csv", header=True, inferSchema=True)

read csv one more time with the same code and you will see that it almostly don't take time, because info already in SparkSession and it will not read nothing
from this file

In [6]:
df.show()

+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|_c0|work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer|   85000|            GBP|       109024|                GB|          50|     

write schema of scv on screen

In [7]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



create schema of this scv

In [8]:
schema = StructType([
    StructField("_c0", IntegerType(), nullable=True),
    StructField("work_year", IntegerType(), nullable=True),
    StructField("experience_level", StringType(), nullable=True),
    StructField("employment_type", StringType(), nullable=True),
    StructField("job_title", StringType(), nullable=True),
    StructField("salary", IntegerType(), nullable=True),
    StructField("salary_currency", StringType(), nullable=True),
    StructField("salary_in_usd", IntegerType(), nullable=True),
    StructField("employee_residence", StringType(), nullable=True),
    StructField("remote_ratio", IntegerType(), nullable=True),
    StructField("company_location", StringType(), nullable=True),
    StructField("company_size", StringType(), nullable=True)
])

restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType.... 
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [9]:
df = spark.read.csv("ds_salaries.csv", header=True, schema=schema)

this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [10]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



now continue to work with one of the dataframes that you create

print data in dataframe using df.show

In [11]:
df.show()

+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|_c0|work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer|   85000|            GBP|       109024|                GB|          50|     

print data in dataframe using display(df.toPandas())

In [12]:
df.toPandas()

,_c0,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...,...
602,602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


create df_job_title that consists from all job_titles without duplicates

In [13]:
df_job_title = df.select(col("job_title")).distinct()

print all rows from df_job_titles without truncating jobs

In [14]:
df_job_title.show(2000, truncate=False)

+----------------------------------------+
|job_title                               |
+----------------------------------------+
|3D Computer Vision Researcher           |
|Lead Data Engineer                      |
|Head of Machine Learning                |
|Data Specialist                         |
|Data Analytics Lead                     |
|Machine Learning Scientist              |
|Lead Data Analyst                       |
|Data Engineering Manager                |
|Staff Data Scientist                    |
|ETL Developer                           |
|Director of Data Engineering            |
|Product Data Analyst                    |
|Principal Data Scientist                |
|AI Scientist                            |
|Director of Data Science                |
|Machine Learning Engineer               |
|Lead Data Scientist                     |
|Machine Learning Infrastructure Engineer|
|Data Science Engineer                   |
|Machine Learning Manager                |
|Research S

create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [15]:
df_analytic = df.groupBy("job_title").agg(
    avg("salary").alias("avg_salary"),
    min("salary").alias("min_salary"),
    max("salary").alias("max_salary")
)

print all rows from df_analytic without trancating jobs

In [16]:
df_analytic.show(100,truncate = False)

+----------------------------------------+------------------+----------+----------+
|job_title                               |avg_salary        |min_salary|max_salary|
+----------------------------------------+------------------+----------+----------+
|3D Computer Vision Researcher           |400000.0          |400000    |400000    |
|Lead Data Engineer                      |140333.33333333334|56000     |276000    |
|Head of Machine Learning                |6000000.0         |6000000   |6000000   |
|Data Specialist                         |165000.0          |165000    |165000    |
|Data Analytics Lead                     |405000.0          |405000    |405000    |
|Machine Learning Scientist              |158412.5          |12000     |260000    |
|Lead Data Analyst                       |569000.0          |87000     |1450000   |
|Data Engineering Manager                |119799.8          |51999     |174000    |
|Staff Data Scientist                    |105000.0          |105000    |1050

now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

In [17]:
window_spec = Window.orderBy(F.desc("avg_salary"))

In [18]:
df_analytic = df_analytic.withColumn("row_id", F.row_number().over(window_spec))

print all data from df_analytic

In [19]:
df_analytic.show(100, truncate = False)

+----------------------------------------+------------------+----------+----------+------+
|job_title                               |avg_salary        |min_salary|max_salary|row_id|
+----------------------------------------+------------------+----------+----------+------+
|Head of Machine Learning                |6000000.0         |6000000   |6000000   |1     |
|ML Engineer                             |2676666.6666666665|14000     |8500000   |2     |
|BI Data Analyst                         |1902045.3333333333|9272      |11000000  |3     |
|Lead Data Scientist                     |1101666.6666666667|115000    |3000000   |4     |
|Data Science Manager                    |1062598.5833333333|137141    |7000000   |5     |
|Lead Data Analyst                       |569000.0          |87000     |1450000   |6     |
|Data Scientist                          |508347.2027972028 |4000      |30400000  |7     |
|Big Data Engineer                       |455000.0          |18000     |1672000   |8     |

it isn't beautifull, so we need to put now row_id on first place in df_analytic

In [20]:
df_analytic = df_analytic.select(col("row_id"), *df_analytic.columns[:-1])

print df_analytic now

In [21]:
df_analytic.show(100, truncate = False)

+------+----------------------------------------+------------------+----------+----------+
|row_id|job_title                               |avg_salary        |min_salary|max_salary|
+------+----------------------------------------+------------------+----------+----------+
|1     |Head of Machine Learning                |6000000.0         |6000000   |6000000   |
|2     |ML Engineer                             |2676666.6666666665|14000     |8500000   |
|3     |BI Data Analyst                         |1902045.3333333333|9272      |11000000  |
|4     |Lead Data Scientist                     |1101666.6666666667|115000    |3000000   |
|5     |Data Science Manager                    |1062598.5833333333|137141    |7000000   |
|6     |Lead Data Analyst                       |569000.0          |87000     |1450000   |
|7     |Data Scientist                          |508347.2027972028 |4000      |30400000  |
|8     |Big Data Engineer                       |455000.0          |18000     |1672000   |

here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

In [22]:
window_spec = Window.partitionBy("experience_level").orderBy(col("salary_in_usd").desc())

print here df_exp_lvl

In [23]:
df_exp_lvl = df.withColumn("rn", row_number().over(window_spec)) \
                        .filter(col("rn") == 1) \
                        .drop("rn") \
                        .withColumnRenamed("salary_in_usd", "biggest_salary")

In [24]:
df_exp_lvl.toPandas()

,_c0,work_year,experience_level,employment_type,job_title,salary,salary_currency,biggest_salary,employee_residence,remote_ratio,company_location,company_size
0,37,2020,EN,FT,Machine Learning Engineer,250000,USD,250000,US,50,US,L
1,252,2021,EX,FT,Principal Data Engineer,600000,USD,600000,US,100,US,L
2,33,2020,MI,FT,Research Scientist,450000,USD,450000,US,0,US,M
3,63,2020,SE,FT,Data Scientist,412000,USD,412000,US,100,US,L


In [25]:
df.toPandas()

,_c0,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...,...
602,602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: id, experience_level, biggest_salary, employee_residence

print df_best

In [26]:
df_best = df.alias("df").join(df_exp_lvl.alias("df_exp"), col("df.salary_in_usd") == col("df_exp.biggest_salary"), "inner").select(col("df._c0"), col("df.experience_level"), col("df_exp.biggest_salary"), col("df.employee_residence"))

In [27]:
df_best.toPandas()

,_c0,experience_level,biggest_salary,employee_residence
0,33,MI,450000,US
1,37,EN,250000,US
2,63,SE,412000,US
3,97,MI,450000,US
4,167,EX,250000,US
5,252,EX,600000,US


drop duplicates if exist by experience_level

In [28]:
df_best = df_best.dropDuplicates(["experience_level"])

print df_best

In [29]:
df_best.toPandas()

,_c0,experience_level,biggest_salary,employee_residence
0,37,EN,250000,US
1,167,EX,250000,US
2,33,MI,450000,US
3,63,SE,412000,US


create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

In [30]:
df_new_best = df_best.drop("_c0").withColumn("level_description", when(col("experience_level") == "MI", "middle").
when(col("experience_level") == "SE", "senior").otherwise(None))

print df_new_best

In [31]:
df_new_best.toPandas()

,experience_level,biggest_salary,employee_residence,level_description
0,EN,250000,US,None
1,EX,250000,US,None
2,MI,450000,US,middle
3,SE,412000,US,senior


write df_new_best like 1.csv and load then it to df_final

In [33]:
df_new_best.write.csv("1.csv", header=True)

print df_final

In [34]:
df_final = spark.read.csv("1.csv", header=True, inferSchema=True)

In [35]:
df_final.show()

+----------------+--------------+------------------+-----------------+
|experience_level|biggest_salary|employee_residence|level_description|
+----------------+--------------+------------------+-----------------+
|              EN|        250000|                US|             null|
|              EX|        250000|                US|             null|
|              MI|        450000|                US|           middle|
|              SE|        412000|                US|           senior|
+----------------+--------------+------------------+-----------------+



filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

In [37]:
df_final = df_final.filter(col("experience_level").isNotNull())

In [43]:
df_final = df.join(df_final, (df["salary_in_usd"] == df_final["biggest_salary"]) & (df["employee_residence"] == df_final["employee_residence"]), "inner")

print df_final

In [44]:
df_final.show()

+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+----------------+--------------+------------------+-----------------+
|_c0|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|experience_level|biggest_salary|employee_residence|level_description|
+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+----------------+--------------+------------------+-----------------+
| 33|     2020|              MI|             FT|  Research Scientist|450000|            USD|       450000|                US|           0|              US|           M|              MI|        450000|                US|           middle|
| 37|     2020|              EN|             FT|

In [45]:
df_final.toPandas()

,_c0,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,experience_level,biggest_salary,employee_residence,level_description
0,33,2020,MI,FT,Research Scientist,450000,USD,450000,US,0,US,M,MI,450000,US,middle
1,37,2020,EN,FT,Machine Learning Engineer,250000,USD,250000,US,50,US,L,EX,250000,US,None
2,37,2020,EN,FT,Machine Learning Engineer,250000,USD,250000,US,50,US,L,EN,250000,US,None
3,63,2020,SE,FT,Data Scientist,412000,USD,412000,US,100,US,L,SE,412000,US,senior
4,97,2021,MI,FT,Financial Data Analyst,450000,USD,450000,US,100,US,L,MI,450000,US,middle
5,167,2021,EX,FT,Director of Data Science,250000,USD,250000,US,0,US,L,EX,250000,US,None
6,167,2021,EX,FT,Director of Data Science,250000,USD,250000,US,0,US,L,EN,250000,US,None


last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

In [48]:
max_salary = df_final.select(max("salary_in_usd")).first()[0]
# The first() function is used to retrieve the maximum value as a Row object, 
# and [0] is used to extract the value from the Row. Finally, we print the maximum salary.

In [49]:
print("Max_salary from final table is: ", max_salary)

Max_salary from final table is:  450000


It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system